To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
%%capture
!pip install --no-deps git+https://github.com/huggingface/transformers.git # Need main branch for Liquid LFM2 models
!pip install --no-deps causal-conv1d==1.5.0.post8 # Install Mamba kernels

In [ ]:
import os

HF_TOKEN = os.environ.get("HF_TOKEN")  # pulled from your Colab secrets
print("✅ Hugging Face token loaded")


### Unsloth

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/LFM2-1.2B-unsloth-bnb-4bit",
    "unsloth/LFM2-700M-unsloth-bnb-4bit",
    "unsloth/LFM2-350M-unsloth-bnb-4bit",

    # Full 16bit unquantized models
    "unsloth/LFM2-1.2B",
    "unsloth/LFM2-700M",
    "unsloth/LFM2-350M",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/LFM2-1.2B-unsloth-bnb-4bit",
    dtype = None, # None for auto detection
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.4: Fast Lfm2 patching. Transformers: 4.57.0.dev0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/209 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # LFM for now is just text only
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `LFM` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. LFM renders multi turn conversations like below:

```
<|startoftext|><|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>
```

In [ ]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Hello!"},
    {"role" : "assistant", "content" : "Hey there!"}
], tokenize = False)

'<|startoftext|><|im_start|>user\nHello!<|im_end|>\n<|im_start|>assistant\nHey there!<|im_end|>\n'

We get the first 3000 rows of the dataset

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("mlabonne/FineTome-100k", split = "train[:3000]")
import os
from google.colab import drive
from datasets import Dataset
import pandas as pd

drive.mount('/content/drive')
save_dir = "/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model"
save_dir_lora = "/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/lora_model"
save_dir_model = "/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/uae_rule_book_QA_assistant"

# Load your dataset
jsonl_path = "/content/drive/My Drive/Colab Notebooks/FineTune_LLM/data/qa_merged_50.jsonl"
df = pd.read_json(jsonl_path, lines=True)

# Combine question and context for the prompt
df["system"] = ""
df["user"] = df["question"]
df["assistant"] = df["answer"]

# Keep only the required columns
df = df[["system", "user", "assistant"]]

# Convert to HuggingFace Dataset
hf_ds = Dataset.from_pandas(df)

Mounted at /content/drive


In [ ]:
# Add conversations field in the expected format
def to_conversation(example):
    return {
        "conversations": [
            {"role": "system", "content": example["system"]},
            {"role": "user", "content": example["user"]},
            {"role": "assistant", "content": example["assistant"]},
        ]
    }

# Apply row-wise
hf_ds = hf_ds.map(to_conversation)


Map:   0%|          | 0/168711 [00:00<?, ? examples/s]

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(hf_ds)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/168711 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [ ]:
dataset[100]

{'system': '',
 'user': 'Can parties request to appear in person during remote communication procedures?',
 'assistant': "Yes, disputing parties in remote communication procedures may request the Committee for personal appearance for hearing of statements at any phase of the Committee's operation.",
 'conversations': [{'content': '', 'role': 'system'},
  {'content': 'Can parties request to appear in person during remote communication procedures?',
   'role': 'user'},
  {'content': "Yes, disputing parties in remote communication procedures may request the Committee for personal appearance for hearing of statements at any phase of the Committee's operation.",
   'role': 'assistant'}]}

We now have to apply the chat template for `LFM` onto the conversations, and save it to `text`. We also remove the BOS token otherwise we'll get double BOS tokens!

In [ ]:
print(len(dataset))

168711


In [ ]:
def formatting_prompts_func(examples):
    texts = tokenizer.apply_chat_template(
        examples["conversations"],
        tokenize = False,
        add_generation_prompt = False,
    )
    return { "text" : [x.removeprefix(tokenizer.bos_token) for x in texts] }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/168711 [00:00<?, ? examples/s]

In [ ]:
dataset[0]["text"]

"<|im_start|>system\n<|im_end|>\n<|im_start|>user\nWhat is the title of the CBUAE Rulebook section provided?<|im_end|>\n<|im_start|>assistant\nThe title of the CBUAE Rulebook section is 'Entire Section'.<|im_end|>\n"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/168711 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=12):   0%|          | 0/168711 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<|startoftext|><|im_start|>system\n<|im_end|>\n<|im_start|>user\nCan parties request to appear in person during remote communication procedures?<|im_end|>\n<|im_start|>assistant\nYes, disputing parties in remote communication procedures may request the Committee for personal appearance for hearing of statements at any phase of the Committee's operation.<|im_end|>\n"

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                          Yes, disputing parties in remote communication procedures may request the Committee for personal appearance for hearing of statements at any phase of the Committee's operation.<|im_end|>\n"

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
1.133 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 168,711 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,142,272 of 1,179,482,880 (0.78% trained)


Step,Training Loss
1,2.912000
2,3.366300
3,2.391300
4,3.144300
5,2.891800
6,2.172400
7,2.514900
8,2.114900
9,2.160300
10,2.902600


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

71.5663 seconds used for training.
1.19 minutes used for training.
Peak reserved memory = 1.252 GB.
Peak reserved memory for training = 0.119 GB.
Peak reserved memory % of max memory = 3.165 %.
Peak reserved memory for training % of max memory = 0.301 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
messages = [{
    "role": "user",
    "content": "What specific area of law does Federal Law No. 5 of 1985 deal with, as mentioned in the text?",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Federal Law No. 5 of 1985 deals with the regulation of the activities of the Central Bank of the Republic of Kazakhstan.<|im_end|>


In [ ]:
messages = [{
    "role": "user",
    "content": "What is the purpose of Article (1) in this resolution?",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Article (1) of this resolution provides for the establishment of a Committee to examine and report to the Council on the implementation of the resolution.<|im_end|>


In [ ]:
messages = [{
    "role": "user",
    "content": "What is the main subject of Decretal Federal Law No (10) of 2018?.",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The main subject of Decretal Federal Law No. 10 of 2018 is the regulation of the Federal Financial System.<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained(save_dir_lora)  # Local saving
tokenizer.save_pretrained(save_dir_lora)
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/lora_model/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/lora_model/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/lora_model/chat_template.jinja',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    import torch  # ✅ Add this
    from unsloth import FastModel
    from transformers import Lfm2ForCausalLM
    model, tokenizer = FastModel.from_pretrained(
        model_name = save_dir_lora, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = torch.float16,
        auto_model = Lfm2ForCausalLM,
        load_in_4bit = True,
    )
    FastModel.for_inference(model) # Enable native 2x faster inference

messages = [{
    "role": "user",
    "content": "What is mentioned about the minimum capital of a company in Article (3)?",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

==((====))==  Unsloth 2025.9.4: Fast Lfm2 patching. Transformers: 4.57.0.dev0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
The minimum capital of a company is stated to be 100,000 Euros.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if True:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        save_dir_lora, # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True,
    )
    tokenizer = AutoTokenizer.from_pretrained(save_dir_lora)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
from peft import PeftModel
from transformers import Lfm2ForCausalLM, AutoTokenizer
import torch

# Load base model (4-bit quantized, but will be converted to float16 after merge)
base_model = Lfm2ForCausalLM.from_pretrained(
    "unsloth/LFM2-1.2B-unsloth-bnb-4bit",
    device_map="auto",
    torch_dtype=torch.float16,
)

# Load LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    save_dir_lora,  # Your trained adapter directory
    device_map="auto",
)

# Merge LoRA into base model (now it's full float16)
model = model.merge_and_unload()

# Save merged full model
model.save_pretrained(save_dir_model)
tokenizer.save_pretrained(save_dir_model)


`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/uae_rule_book_QA_assistant/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/uae_rule_book_QA_assistant/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/uae_rule_book_QA_assistant/chat_template.jinja',
 '/content/drive/My Drive/Colab Notebooks/FineTune_LLM/model/uae_rule_book_QA_assistant/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Push model
AutoModelForCausalLM.from_pretrained(save_dir_model).push_to_hub("rajeshthangaraj1/uae_rule_book_QA_assistant", token = HF_TOKEN)

# Push tokenizer
AutoTokenizer.from_pretrained(save_dir_model).push_to_hub("rajeshthangaraj1/uae_rule_book_QA_assistant", token = HF_TOKEN)


README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/rajeshthangaraj1/uae_rule_book_QA_assistant/commit/20c891fe306e38042b971ff38b467f8c7e4cfcfb', commit_message='Upload tokenizer', commit_description='', oid='20c891fe306e38042b971ff38b467f8c7e4cfcfb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rajeshthangaraj1/uae_rule_book_QA_assistant', endpoint='https://huggingface.co', repo_type='model', repo_id='rajeshthangaraj1/uae_rule_book_QA_assistant'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your fine-tuned model
model = AutoModelForCausalLM.from_pretrained(
    save_dir_model,
    device_map="auto",
    dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(save_dir_model)

# Question
question = "What is the relationship between Decree Law No. (20) of 2018 and Cabinet Decision No. (10) of 2019?"

# Add strict system instruction
messages = [
    {"role": "system", "content": "You are an assistant specialized in the UAE Central Bank Rulebook. Only answer based on the UAE Rulebook. If the answer is not in the Rulebook, reply 'Not found in UAE Rulebook'."},
    {"role": "user", "content": question}
]

# Apply chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize (remove token_type_ids if present)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
inputs.pop("token_type_ids", None)

# Generate with strict decoding
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.1,   # low = less hallucination
    top_p=0.95,
    do_sample=False,   # greedy decoding
)

# Decode only new tokens
answer = tokenizer.decode(outputs[0], skip_special_tokens=True, skip_prompt=True)
print(answer.strip())


system
You are an assistant specialized in the UAE Central Bank Rulebook. Only answer based on the UAE Rulebook. If the answer is not in the Rulebook, reply 'Not found in UAE Rulebook'.
user
What is the relationship between Decree Law No. (20) of 2018 and Cabinet Decision No. (10) of 2019?
assistant
Decree Law No. (20) of 2018 and Cabinet Decision No. (10) of 2019 are both part of the regulatory framework in the UAE, but they serve different purposes and are not directly related in terms of their legal status. Decree Law No. (20) of 2018 deals with the establishment of the UAE Central Bank's regulatory framework, particularly concerning the supervision and regulation of financial institutions. On the other hand, Cabinet Decision No. (10) of 2019 is a directive issued by the Cabinet to implement specific regulations or guidelines related to the Central Bank's operations as per Decree Law No. (20).

In essence, Decree Law No. (20) sets the overarching regulatory framework, while Cabinet 